# Serving your model exercise. Part 1 - Flask

## Intro
Reminder, there will usually be 3 different places where the code relevant to our model prediction runs:
1. **Training computer / server** - where we train our model and save it
2. **Inference server** - server that listens to REST API requests to make predictions / inferences with the model that was trained on the model server. Potentially, we could have many such servers. 
3. **Client** - client application (browser, mobile app etc.) that needs a prediction, and requests from **inference server** over HTTP with REST API to make the prediction

There are 2 directions for **sending data**:
1. Data sent via a REST API from the **client** to the **Inference server**
1. Prediction result that's sent from the **Inference server** back to the **client**

There are 2 ways to send the data from **client** to the **Inference server**:
1. As parameters of the URL
1. As a body of the HTTP request

Prediction that's returned from the **Inference server** to the **client** will always be in the body of the HTTP (limitation of the HTTP protocol).  However, it could be in different formats - regular string, HTML page, or a JSON file.

In this exercise we will learn about 2 combinations of the above:
1. Sending data as **parameters of the request** (as part of the URL) and receiving data as a **regular string** - useful for small and short data as inputs to the model, and when the prediction is short / simple.  We will use it to implement a **single prediction API**.
1. Sending data in the HTTP body as a **JSON file**, and getting back the prediction as a **JSON file** - more relevant for when the data has many features / complicated features, and when the prediction response is itself slightly longer / more complicated.  We will use it to implement **multiple predictions API**

Of course, there is no connection between the format of data sent to the server, and received back from the server, so you could have other variations.

## 1. Getting to a trained model
- Choose one of the models you trained in one the previous exercises or any other model. **Do not take something from many Flask examples online!**  **For easy debugging** - It's better to use a model with a small number of features and where the feature values are not long arrays (you can also take a small subsete of features of existing model).
- Specify where can one download the dataset from (to be used during checking the exercise)
- Say in one word what is the business problem and what you are predicting 
- Preprocess, split to train and test dataset
- Train the model - how well your models predicts (accuracy / $R^2$) is not of big importance here
- Do a few predictions of the model locally

This code will run on the **training server**

**import packages and modules**

In [1]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import pickle
import json
from flask import Flask, request
import pandas as pd 
import requests

**some constants**

In [2]:
DATA = 'data'
TARGET = 'target'
RANDOM_STATE = 42
TEST_SIZE = 0.2
SYNTHETIC = 3
FEATURE_NAMES = 'feature_names'
MODEL_FILE_NAME = 'tyra_banks.pkl'
JSON_TEST_FILE_NAME = 'statham.json'
JSON_SYNTHETIC = 'voorhees.json'
RECORD_ORIENT = 'records'
PREDICTION_KEY = 'predictions'
JSON_KEY = 'json'

### 1)

- for this task I have chosen to use the **Boston data set** from **sklearn**. this data set is fairly small with 506 samples and 13 features. we have used a version of it previously in class<br>

- the data set can be imported directly with sklearn using `from sklearn.datasets import load_boston`

- in one word I'm trying to predict: **price** (in two words house prices). the business question is - how to predict a house price based on historical data (10 words... sorry)<br><br>**regression** task

**now lets get to work**<br>
a. import data set

In [3]:
boston = load_boston()
X,y = boston[DATA], boston[TARGET] 

b. split to test and train

In [4]:
X_train, X_test, y_train, y_test = \
train_test_split(X, y, test_size=TEST_SIZE, 
                 random_state=RANDOM_STATE)

c. train **RandomForestRegressor** with default parameters

In [5]:
rfr = RandomForestRegressor(random_state=RANDOM_STATE)
rfr.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

d. predict and evaluate

In [6]:
y_pred = rfr.predict(X_test)
print('RMSE score: {}'.format(np.round(mean_squared_error(y_test, y_pred, squared=False), 2)))
print('R^2 score: {}'.format(np.round(r2_score(y_test, y_pred), 2)))

RMSE score: 2.81
R^2 score: 0.89


e. local predictions:

1. generated new testings using function that creates a synthetic data based of randomizing values from each of the train features
2. predict the newly generated records
        1.

In [7]:
def get_record(x, n):
    records = []
    for _ in range(n):
        records.append(np.array(
            list(map(
                lambda idx: np.random.choice(x[:, idx]), 
                range(len(x[0,:]))))).reshape(1,-1))
    return np.concatenate(records)

synthetic_x = get_record(X_train, SYNTHETIC)
print('newly generated synthetic records')
pd.DataFrame(synthetic_x, columns=boston[FEATURE_NAMES])

newly generated synthetic records


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,2.37857,25.0,9.90,0.0,0.547,6.421,85.7,1.5184,5.0,666.0,17.9,391.43,24.10
1,16.81180,0.0,2.89,0.0,0.504,6.012,53.6,1.7523,4.0,289.0,16.0,393.74,4.84
2,10.83420,20.0,18.10,0.0,0.671,4.652,58.7,1.7984,24.0,284.0,19.1,349.48,22.88


        2.

In [8]:
synthetic_y_pred = rfr.predict(synthetic_x)
print(synthetic_y_pred)

[15.447 25.658 12.889]


## 2. Save you model, predict with saved model

Simulate in this notebook code that will happen during training on the **training server**:
- Using `pickle`, save your model to disk. Reference: https://scikit-learn.org/stable/modules/model_persistence.html
- Save the test dataset to file.  What's a good format(s) for saving datasets?

### 2)

**pickling model and stuff**

In [9]:
pickle.dump(rfr, open(MODEL_FILE_NAME, 'wb'))

**jasoning datasets and stuff**

In [10]:
json_test = pd.DataFrame(X_test, columns=boston[FEATURE_NAMES]).to_json(orient=RECORD_ORIENT)
json_synthetic = pd.DataFrame(synthetic_x, columns=boston[FEATURE_NAMES]).to_json(orient=RECORD_ORIENT)

with open(JSON_TEST_FILE_NAME, 'w') as jt:
    jt.write(json_test)
    
with open(JSON_SYNTHETIC, 'w') as js:
    js.write(json_synthetic)

Simulate in this notebook code that will happen during inference on the **inference server**:
- Load the model again with `pickle`.
- Read the test dataset file, and perform some predictions
- Compare the predictions received before saving the model, and after reading a saved model.  Show that you get the same results. 

**loading the model and stuff**

In [11]:
loaded_forest = pickle.load(open(MODEL_FILE_NAME, 'rb'))

**read some jsons and predict**

In [12]:
loaded_test = pd.json_normalize(json.load(open(JSON_TEST_FILE_NAME, 'r')))
loaded_synthetic = pd.json_normalize(json.load(open(JSON_SYNTHETIC, 'r')))

**compare and predict**

In [13]:
print('using serialized test set with pickled model (also serialized) '
      'gives the same predictions?\n{}'. \
      format(np.all(loaded_forest.predict(loaded_test) == y_pred)))

print('using serialized synthetic set with pickled model(also serialized) '
      'gives the same predictions?\n{}'. \
      format(np.all(loaded_forest.predict(synthetic_x) == synthetic_y_pred)))

using seriliezed test set with pickled model (also serilized) gives the same predictions?
True
using seriliezed synthetic set with pickled model(also serilized) gives the same predictions?
True


## 3. Serve your model - using URL parameters

Now we are done with **training server**, since we have the saved model.  From now all that's relevant is **inference server** and **client** code.

Let's create the **inference server** that answers to REST APIs with predictions:

- Using `flask`, create a Pycharm project and implement the following prediction API:
- **Single prediction API** that receives inputs as parameters (no body), and returns a single prediction as a string / text.  
- Example: http://localhost:5000/predict_single?key1=value1&key2=value2 (replace `key1`, `value1` etc. names with your relevant feature names and values) that would return the class label (example: `0` / `1`)
- **Important:** For efficiency purposes, consider what's the best place in your code to put the code that reads the model.  Why?
- **Important:** In general, take runtime efficiency into account.  Your API might be called large number of times per second, and you will be paying for more inference servers if your code is not efficient.
- Copy your **inference server** code also here for reference

### 3)

**copy stuff**

In [14]:
'''
app = Flask(__name__)
model = pickle.load(open(MODEL_FILE_NAME, 'rb'))

@app.route('/predict')
def predict():
    """
    function for predicting house prices by client requests
    :return: predictions in JSON
    """
    data = pd.json_normalize(json.loads(request.args.get(JSON_KEY)))
    prediction = model.predict(data).tolist()
    return json.dumps(prediction)

def main():
    #app.run()
    pass

if __name__ == '__main__':
    main()
'''

'\napp = Flask(__name__)\nmodel = pickle.load(open(MODEL_FILE_NAME, \'rb\'))\n\n@app.route(\'/predict\')\ndef predict():\n    """\n    function for predicting house prices by client requests\n    :return: predictions in JSON\n    """\n    data = pd.json_normalize(json.loads(request.args.get(JSON_KEY)))\n    prediction = model.predict(data).tolist()\n    return json.dumps(prediction)\n\ndef main():\n    #app.run()\n    pass\n\nif __name__ == \'__main__\':\n    main()\n'

## 4. Consume your model with python

#### Simulate client requests for inference / prediction:
Assume your client runs Python code also, and not only your training and inference servers (in real case scenario, often times your client code will actually not be in Python).
Use Python `requests` module from here to request a prediction by the client from to the inference .  To pass parameters with Python `requests` module, use the `params` parameter of `requests.get` API.

**Print input and output of the prediction.**

**Warning**: don't get used to seeing it in a Jupyter notebook.  This code will usually run inside a **client application**

### 4)

**print a prediction input**
    1. test set input - dictionary of JSON key, and a value as JSON (text)

In [15]:
print({JSON_KEY: json_test})

{'json': '[{"CRIM":0.09178,"ZN":0.0,"INDUS":4.05,"CHAS":0.0,"NOX":0.51,"RM":6.416,"AGE":84.1,"DIS":2.6463,"RAD":5.0,"TAX":296.0,"PTRATIO":16.6,"B":395.5,"LSTAT":9.04},{"CRIM":0.05644,"ZN":40.0,"INDUS":6.41,"CHAS":1.0,"NOX":0.447,"RM":6.758,"AGE":32.9,"DIS":4.0776,"RAD":4.0,"TAX":254.0,"PTRATIO":17.6,"B":396.9,"LSTAT":3.53},{"CRIM":0.10574,"ZN":0.0,"INDUS":27.74,"CHAS":0.0,"NOX":0.609,"RM":5.983,"AGE":98.8,"DIS":1.8681,"RAD":4.0,"TAX":711.0,"PTRATIO":20.1,"B":390.11,"LSTAT":18.07},{"CRIM":0.09164,"ZN":0.0,"INDUS":10.81,"CHAS":0.0,"NOX":0.413,"RM":6.065,"AGE":7.8,"DIS":5.2873,"RAD":4.0,"TAX":305.0,"PTRATIO":19.2,"B":390.91,"LSTAT":5.52},{"CRIM":5.09017,"ZN":0.0,"INDUS":18.1,"CHAS":0.0,"NOX":0.713,"RM":6.297,"AGE":91.8,"DIS":2.3682,"RAD":24.0,"TAX":666.0,"PTRATIO":20.2,"B":385.09,"LSTAT":17.27},{"CRIM":0.10153,"ZN":0.0,"INDUS":12.83,"CHAS":0.0,"NOX":0.437,"RM":6.279,"AGE":74.5,"DIS":4.0522,"RAD":5.0,"TAX":398.0,"PTRATIO":18.7,"B":373.66,"LSTAT":11.97},{"CRIM":0.31827,"ZN":0.0,"INDUS":9.9,

    2. synthetic input (3 records) - dictionary of JSON key and a value as JSON (text)


In [16]:
print({JSON_KEY: json_synthetic})

{'json': '[{"CRIM":2.37857,"ZN":25.0,"INDUS":9.9,"CHAS":0.0,"NOX":0.547,"RM":6.421,"AGE":85.7,"DIS":1.5184,"RAD":5.0,"TAX":666.0,"PTRATIO":17.9,"B":391.43,"LSTAT":24.1},{"CRIM":16.8118,"ZN":0.0,"INDUS":2.89,"CHAS":0.0,"NOX":0.504,"RM":6.012,"AGE":53.6,"DIS":1.7523,"RAD":4.0,"TAX":289.0,"PTRATIO":16.0,"B":393.74,"LSTAT":4.84},{"CRIM":10.8342,"ZN":20.0,"INDUS":18.1,"CHAS":0.0,"NOX":0.671,"RM":4.652,"AGE":58.7,"DIS":1.7984,"RAD":24.0,"TAX":284.0,"PTRATIO":19.1,"B":349.48,"LSTAT":22.88}]'}


**predicting**

(+minimal parsing - since the client has python it should work for him/her)

In [17]:
url = 'http://localhost:5000/predict'
y_pred = json.loads(requests.get(url=url, params={JSON_KEY: json_test}).text)[PREDICTION_KEY]
synthetic_y_pred = json.loads(requests.get(url=url, params={JSON_KEY: json_synthetic}).text)[PREDICTION_KEY]

**printing prediction output**

    1. test set prediction

In [18]:
y_pred

[22.839000000000002,
 30.67599999999999,
 16.317,
 23.51,
 16.819000000000006,
 21.373999999999995,
 19.358,
 15.619999999999987,
 21.09100000000001,
 21.07300000000001,
 20.046999999999997,
 19.297000000000004,
 8.610999999999992,
 21.397999999999996,
 19.378000000000004,
 25.452999999999992,
 19.187,
 8.537999999999998,
 46.13200000000003,
 14.535999999999987,
 24.728,
 23.996000000000002,
 14.508999999999991,
 23.846999999999998,
 14.363000000000001,
 14.796,
 21.125999999999998,
 13.663,
 19.534999999999997,
 21.290000000000006,
 19.448999999999998,
 23.392999999999983,
 29.300000000000008,
 20.337999999999994,
 14.595999999999993,
 15.593999999999989,
 33.834999999999994,
 19.122999999999998,
 20.91500000000001,
 24.375999999999998,
 19.28599999999999,
 29.61,
 46.10800000000002,
 19.427999999999997,
 22.652999999999988,
 13.675999999999995,
 15.034999999999993,
 24.32099999999999,
 18.689,
 28.820999999999984,
 21.107000000000014,
 33.81099999999998,
 16.501999999999995,
 25.7789

    2. synthetic set prediction

In [19]:
synthetic_y_pred

[15.446999999999996, 25.657999999999987, 12.889]

**sanity check**

In [20]:
print('using serialized test set with pickled model (also serialized) '
      'on an inference server\ngives the same predictions?\n{}'. \
      format(np.all(loaded_forest.predict(loaded_test) == np.array(y_pred))))

print('using serialized synthetic set with pickled model(also serialized) '
      'on an inference server\ngives the same predictions?\n{}'. \
      format(np.all(loaded_forest.predict(synthetic_x) == np.array(synthetic_y_pred))))

using seriliezed test set with pickled model (also serilized) on an inference server
gives the same predictions?
True
using seriliezed synthetic set with pickled model(also serilized) on an inference server
gives the same predictions?
True


**COMMENT**

okay just read Q5 and only now I understand (gotta admit that it wasn't 100% clear) that in a previous section you probably wanted me to write a manual query with a lot of keys and a lot of values and I kind of *hacked* it using one key **'json'**, and a string value that my inference server uses to extract the information from - but come on it is cooler that way, and the first method is very boring with a lot of unnecessary work, isn't it?

## 5. Serve your model - using JSON files

- Using `flask`, add code to your previous file in Pycharm **with inference code** to create the following prediction API (in addition to **Single prediction API** done above):
- **Multiple prediction API** that receives input many observations to predict on as a json file in the body, and returns a json file with predictions.
- Your **JSON** file format has to be efficient, clear and following JSON file syntax: 
  - JSON file is a nested structure of potentially multiple dictionaries and lists 
  - JSON file tip: Use lists, every member in the list can be a dictionary of all the features.  
  - JSON file tip: Do not put indexes of predictions into the JSON files, indexes of predictions can be easily computed with Python code later 
  - JSON files are sometimes slightly verbose, but are extremely human readable.  Just looking at your JSON files of input and output, is it possible to understand what were the observations in input and what were the predictions in output?
  - See https://www.json.org/json-en.html for JSON format
- Think about efficiency of your code - your REST API might be called a huge number of times, with a huge number of observations every time.  Can part of the code be done only once?  Can you predict on everything together? Can you do less or cheaper data conversions?
- Example of URL that will be used to predict: http://localhost:5000/car-price
- Reference for working with JSONs in Flask: https://pythonise.com/series/learning-flask/working-with-json-in-flask
- Do you need a GET or a POST type of REST API call? Does it change what you did in step 3?  Conceptually, would you say it makes sense to use GET or POST types for predictions?
- Copy your **inference server** code also here for reference 

### 5)

**Answers**

1. it seems to that both the input and output JSONs are very clear regarding understanding the meanings
2. the REST API can analyze several records at once. regarding fewer data conversions - I guess I can do without the pandas json normalize (but it is so convenient - don't think that parsing would be cheaper)
3. from inference server side I'm using GET the costumer would use POST - conceptually it depends on who we are speaking about as a client it would make sense to use POST and wait for a reply as for the server it would make sense to use GET

**now for copying more stuff**

In [21]:
'''
@app.route('/predict')
def predict():
    """
    function for predicting house prices by client requests
    :return: predictions in JSON
    """
    data = pd.json_normalize(json.loads(request.args.get(JSON_KEY)))
    prediction = {PREDICTION_KEY: model.predict(data).tolist()}
    return json.dumps(prediction)


@app.route('/predict_json', methods=['POST'])
def predict_json():
    """
    function for predicting house prices using client json input
    :return: predictions in JSON
    """
    data = pd.json_normalize(json.loads(request.get_json()))
    prediction = {PREDICTION_KEY: model.predict(data).tolist()}
    return json.dumps(prediction)


def main():
    app.run()
    pass


if __name__ == '__main__':
    main()
'''

'\n@app.route(\'/predict\')\ndef predict():\n    """\n    function for predicting house prices by client requests\n    :return: predictions in JSON\n    """\n    data = pd.json_normalize(json.loads(request.args.get(JSON_KEY)))\n    prediction = {PREDICTION_KEY: model.predict(data).tolist()}\n    return json.dumps(prediction)\n\n\n@app.route(\'/predict_json\', methods=[\'POST\'])\ndef predict_json():\n    """\n    function for predicting house prices using client json input\n    :return: predictions in JSON\n    """\n    data = pd.json_normalize(json.loads(request.get_json()))\n    prediction = {PREDICTION_KEY: model.predict(data).tolist()}\n    return json.dumps(prediction)\n\n\ndef main():\n    app.run()\n    pass\n\n\nif __name__ == \'__main__\':\n    main()\n'

Use Python `requests` module from here to make a prediction, and **print the input, and the output** of the prediction (or part of it if it's too large).  

**Hint:** to pass a JSON file to the `requests` module, use `json` parameter of the `requests.post` API.

**Warning**: don't get used to seeing it in a Jupyter notebook.  This code will usually run inside a **client application**

**print a prediction input**
    1. test set input - JSON serialized text 

In [22]:
json_test

'[{"CRIM":0.09178,"ZN":0.0,"INDUS":4.05,"CHAS":0.0,"NOX":0.51,"RM":6.416,"AGE":84.1,"DIS":2.6463,"RAD":5.0,"TAX":296.0,"PTRATIO":16.6,"B":395.5,"LSTAT":9.04},{"CRIM":0.05644,"ZN":40.0,"INDUS":6.41,"CHAS":1.0,"NOX":0.447,"RM":6.758,"AGE":32.9,"DIS":4.0776,"RAD":4.0,"TAX":254.0,"PTRATIO":17.6,"B":396.9,"LSTAT":3.53},{"CRIM":0.10574,"ZN":0.0,"INDUS":27.74,"CHAS":0.0,"NOX":0.609,"RM":5.983,"AGE":98.8,"DIS":1.8681,"RAD":4.0,"TAX":711.0,"PTRATIO":20.1,"B":390.11,"LSTAT":18.07},{"CRIM":0.09164,"ZN":0.0,"INDUS":10.81,"CHAS":0.0,"NOX":0.413,"RM":6.065,"AGE":7.8,"DIS":5.2873,"RAD":4.0,"TAX":305.0,"PTRATIO":19.2,"B":390.91,"LSTAT":5.52},{"CRIM":5.09017,"ZN":0.0,"INDUS":18.1,"CHAS":0.0,"NOX":0.713,"RM":6.297,"AGE":91.8,"DIS":2.3682,"RAD":24.0,"TAX":666.0,"PTRATIO":20.2,"B":385.09,"LSTAT":17.27},{"CRIM":0.10153,"ZN":0.0,"INDUS":12.83,"CHAS":0.0,"NOX":0.437,"RM":6.279,"AGE":74.5,"DIS":4.0522,"RAD":5.0,"TAX":398.0,"PTRATIO":18.7,"B":373.66,"LSTAT":11.97},{"CRIM":0.31827,"ZN":0.0,"INDUS":9.9,"CHAS":0.

    2. synthetic input (3 records) - JSON serialized text

In [23]:
json_synthetic

'[{"CRIM":2.37857,"ZN":25.0,"INDUS":9.9,"CHAS":0.0,"NOX":0.547,"RM":6.421,"AGE":85.7,"DIS":1.5184,"RAD":5.0,"TAX":666.0,"PTRATIO":17.9,"B":391.43,"LSTAT":24.1},{"CRIM":16.8118,"ZN":0.0,"INDUS":2.89,"CHAS":0.0,"NOX":0.504,"RM":6.012,"AGE":53.6,"DIS":1.7523,"RAD":4.0,"TAX":289.0,"PTRATIO":16.0,"B":393.74,"LSTAT":4.84},{"CRIM":10.8342,"ZN":20.0,"INDUS":18.1,"CHAS":0.0,"NOX":0.671,"RM":4.652,"AGE":58.7,"DIS":1.7984,"RAD":24.0,"TAX":284.0,"PTRATIO":19.1,"B":349.48,"LSTAT":22.88}]'

**predicting**

In [24]:
url = 'http://localhost:5000/predict_json'
y_pred = requests.post(url=url, json=json_test)
synthetic_y_pred = requests.post(url=url, json=json_synthetic)

**printing prediction output**

    1. test set prediction

In [25]:
y_pred.text

'{"predictions": [22.839000000000002, 30.67599999999999, 16.317, 23.51, 16.819000000000006, 21.373999999999995, 19.358, 15.619999999999987, 21.09100000000001, 21.07300000000001, 20.046999999999997, 19.297000000000004, 8.610999999999992, 21.397999999999996, 19.378000000000004, 25.452999999999992, 19.187, 8.537999999999998, 46.13200000000003, 14.535999999999987, 24.728, 23.996000000000002, 14.508999999999991, 23.846999999999998, 14.363000000000001, 14.796, 21.125999999999998, 13.663, 19.534999999999997, 21.290000000000006, 19.448999999999998, 23.392999999999983, 29.300000000000008, 20.337999999999994, 14.595999999999993, 15.593999999999989, 33.834999999999994, 19.122999999999998, 20.91500000000001, 24.375999999999998, 19.28599999999999, 29.61, 46.10800000000002, 19.427999999999997, 22.652999999999988, 13.675999999999995, 15.034999999999993, 24.32099999999999, 18.689, 28.820999999999984, 21.107000000000014, 33.81099999999998, 16.501999999999995, 25.778999999999996, 44.922000000000004, 21.

    2. synthetic set prediction

In [26]:
synthetic_y_pred.text

'{"predictions": [15.446999999999996, 25.657999999999987, 12.889]}'

**a bit of parsing**

In [27]:
y_pred = json.loads(y_pred.text)[PREDICTION_KEY]
synthetic_y_pred = json.loads(synthetic_y_pred.text)[PREDICTION_KEY]

**sanity check**

In [28]:
print('using serialized test set with pickled model (also serialized) '
      'on an inference server\n with a JSON input gives '
      'the same predictions?\n{}'.
      format(np.all(loaded_forest.predict(X_test).tolist() == y_pred)))

print('using serialized synthetic set with pickled model(also serialized) '
      'on an inference server\n with a JSON input gives the same '
      'predictions?\n{}'.
      format(np.all(loaded_forest.predict(synthetic_x).tolist() == synthetic_y_pred)))

using seriliezed test set with pickled model (also serilized) on an inference server
 with a JSON input gives the same predictions?
True
using seriliezed synthetic set with pickled model(also serilized) on an inference server
 with a JSON input gives the same predictions?
True


## 6. Submit a zip file with:
1. This notebook
2. Your Python inference server file